# Lab 9: RNNs

Authors: Omar, Rupal, Van

### Context

The SMS Spam Collection is a set of SMS tagged messages that have been collected for SMS Spam research. It contains one set of SMS messages in English of 5,574 messages, tagged acording being ham (legitimate) or spam. Data was originally used to detect SMS spam.

### Content

The files contain one message per line. Each line is composed by two columns: v1 contains the label (ham or spam) and v2 contains the raw text.

### Acknowledgements

The original dataset can be found here. The creators would like to note that in case you find the dataset useful, please make a reference to previous paper and the web page: http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/ in your papers, research, etc.

### Classification problem

Can we use this dataset to build a prediction model that will accurately classify which texts are spam?


## Data preprocessing

In [17]:
import pandas as pd
import numpy as np
df = pd.read_csv('spam.csv', encoding='latin1')
# drop blank columns
df = df.drop('Unnamed: 2',1)
df = df.drop('Unnamed: 3',1)
df = df.drop('Unnamed: 4',1)

In [18]:
# v1 is target v2 is the text
columns = ['v1','v2']
df_bin = pd.DataFrame(columns = columns)
end = df.size - 2
# change v1 from spam ham to binary 0 1
for i in range(0,5572):
    if(df['v1'][i] == "ham"):
        df['v1'][i] = 0
    else:
        df['v1'][i] = 1

In [19]:
X = df['v2']
y = df['v1']
# still need to encode y for karis
print(X.shape, y.shape)

(5572,) (5572,)


In [20]:
# max chars
max_text_length = 0
max_text = ''
for text in X:
    if len(text) > max_text_length:
        max_text_length = len(text)
        max_text = text
print(max_text_length, max_text)

910 For me the love should start with attraction.i should feel that I need her every time around me.she should be the first thing which comes in my thoughts.I would start the day and end it with her.she should be there every time I dream.love will be then when my every breath has her name.my life should happen around her.my life will be named to her.I would cry for her.will give all my happiness and take all her sorrows.I will be ready to fight with anyone for her.I will be in love when I will be doing the craziest things for her.love will be when I don't have to proove anyone that my girl is the most beautiful lady on the whole planet.I will always be singing praises for her.love will be when I start up making chicken curry and end up makiing sambar.life will be the most beautiful then.will get every morning and thank god for the day because she is with me.I would like to say a lot..will tell later..


_____

# RNNs with Pre-processing in Keras

## Moving to 20 Newsgroups
So we should be able to convert a new dataset into the same format as above. Let's do this from scratch, converting the 20 news groups data. 
- http://qwone.com/~jason/20Newsgroups/

We looked at this a while back when we created the tf-idf and bag-of-words models. This time, we are not going to get rid of the sequence of words for classification. 

In [21]:
%%time
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

NUM_TOP_WORDS = None
MAX_ART_LEN = max_text_length # maximum and minimum number of words

tokenizer = Tokenizer(num_words=NUM_TOP_WORDS)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index
NUM_TOP_WORDS = len(word_index) if NUM_TOP_WORDS==None else NUM_TOP_WORDS
top_words = min((len(word_index),NUM_TOP_WORDS))
print('Found %s unique tokens. Distilled to %d top words.' % (len(word_index),top_words))

X = pad_sequences(sequences, maxlen=MAX_ART_LEN)

y_ohe = keras.utils.to_categorical(y)
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', y_ohe.shape)
print(np.max(X))

Found 8920 unique tokens. Distilled to 8920 top words.
Shape of data tensor: (5572, 910)
Shape of label tensor: (5572, 2)
8920
Wall time: 304 ms


In [22]:
print(type(X))
print(X.shape)
print(type(y_ohe))
print(y_ohe.shape)
print(type(y))
y_matrix = y.as_matrix
print(type(y_matrix))

<class 'numpy.ndarray'>
(5572, 910)
<class 'numpy.ndarray'>
(5572, 2)
<class 'pandas.core.series.Series'>
<class 'method'>


So that's it! The representation is now:
- each word is converted to an integer 
- each article is a series of integers that represent the correct ordering of words
- the target is one hot encoded

___

In [23]:
from sklearn.model_selection import train_test_split
# Split it into train / test subsets
X_train, X_test, y_train_ohe, y_test_ohe = train_test_split(X, y_ohe, test_size=0.2,
                                                            stratify=y, 
                                                            random_state=42)
NUM_CLASSES = 2
print(X_train.shape,y_train_ohe.shape)
print(np.sum(y_train_ohe,axis=0))

(4457, 910) (4457, 2)
[ 3859.   598.]


## Loading the embedding
But this is going to be a more involved process. Maybe we can speed up the training by loading up a pre-trained embedding of the words?!

Let's use the GloVe word embedding in keras. We will follow the example at:
- https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

You can download the nearly 1GB pretrained embeddings here:
- http://nlp.stanford.edu/data/glove.6B.zip

Let's take a quick look at the format of the file:

In [25]:
%%time
EMBED_SIZE = 100
# the embed size should match the file you load glove from
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf8')
# save key/array pairs of the embeddings
#  the key of the dictionary is the word, the array is the embedding
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# now fill in the matrix, using the ordering from the
#  keras word tokenizer from before
embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

Found 400000 word vectors.
(8921, 100)
Wall time: 17.5 s


In [26]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBED_SIZE,
                            weights=[embedding_matrix],
                            input_length=MAX_ART_LEN,
                            trainable=False)

In [27]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

rnn = Sequential()
rnn.add(embedding_layer)
rnn.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
rnn.add(Dense(NUM_CLASSES, activation='sigmoid'))
rnn.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(rnn.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 910, 100)          892100    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 972,702.0
Trainable params: 80,602.0
Non-trainable params: 892,100.0
_________________________________________________________________
None


In [28]:
rnn.fit(X_train, y_train_ohe, validation_data=(X_test, y_test_ohe), epochs=3, batch_size=64)

Train on 4457 samples, validate on 1115 samples
Epoch 1/3
4457/4457 [==============================] - 399s - loss: 0.1875 - acc: 0.9331 - val_loss: 0.1199 - val_acc: 0.9578
Epoch 2/3
4457/4457 [==============================] - 498s - loss: 0.1023 - acc: 0.9688 - val_loss: 0.0959 - val_acc: 0.9641
Epoch 3/3
4457/4457 [==============================] - 497s - loss: 0.0903 - acc: 0.9706 - val_loss: 0.0890 - val_acc: 0.9695


In [29]:
rnn2 = Sequential()
rnn2.add(embedding_layer)
rnn2.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
rnn2.add(Dense(NUM_CLASSES, activation='sigmoid'))
rnn2.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(rnn2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 910, 100)          892100    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 972,702.0
Trainable params: 80,602.0
Non-trainable params: 892,100.0
_________________________________________________________________
None


In [30]:
rnn2.fit(X_train, y_train_ohe, validation_data=(X_test, y_test_ohe), epochs=3, batch_size=64)

Train on 4457 samples, validate on 1115 samples
Epoch 1/3
4457/4457 [==============================] - 471s - loss: 0.2031 - acc: 0.9265 - val_loss: 0.1168 - val_acc: 0.9583
Epoch 2/3
4457/4457 [==============================] - 487s - loss: 0.1085 - acc: 0.9665 - val_loss: 0.1046 - val_acc: 0.9619
Epoch 3/3
4457/4457 [==============================] - 488s - loss: 0.0828 - acc: 0.9736 - val_loss: 0.0833 - val_acc: 0.9704
